In [1]:
import tensorflow as tf
from glob import glob
import os
from sklearn.model_selection import train_test_split
import pandas as pd
import cv2
import numpy as np

In [2]:
def load_data(dataset_path):
    images = sorted(glob(os.path.join("data", "train", "pha", "laptop_a0005_indoor") +"/*"))
    masks = sorted(glob(os.path.join("data\\train\\fgr\\laptop_a0005_indoor") + "/*"))
    images
    return images, masks

In [3]:
def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    x = cv2.resize(x, (256, 256))
    x = x/255.0
    x = x.astype(np.float32)
    # (256, 256, 3)
    return x

In [4]:
def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, (256, 256))
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)
    return x

In [5]:
def preprocess(image_path, mask_path):
    def f(image_path, mask_path):
        image_path = image_path.decode()
        mask_path = mask_path.decode()
        x = read_image(image_path)
        y = read_mask(mask_path)
        return x, y
    image, mask = tf.numpy_function(f, [image_path, mask_path], [tf.float32, tf.float32])
    image.set_shape([256, 256, 3])
    mask.set_shape([256, 256, 1])
    return image, mask

In [6]:
def tf_dataset(images, masks, batch=30):
    dataset = tf.data.Dataset.from_tensor_slices((images, masks))
    dataset = dataset.shuffle(buffer_size=5000)
    dataset = dataset.map(preprocess)
    dataset = dataset.batch(batch)
    dataset = dataset.prefetch(2)
    return dataset

In [7]:
#dataset_path = "../input/person-segmentation/people_segmentation"
dataset_path ="D:\\dissertation\\code\\"
batch_size = 30

In [8]:
 ds =  pd.read_csv("data.csv")

In [9]:
ds

,image,mask
0,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
1,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
2,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
3,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
4,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
...,...,...
145,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
146,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
147,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...
148,D:\dissertation\code\data\train\fgr\laptop_a00...,D:\dissertation\code\data\train\pha\laptop_a00...


In [10]:
#train_x, train_y = load_data("dataset_path")

train_dataset = tf_dataset(ds['image'].tolist(), ds['mask'].tolist(), batch=batch_size)
#valid_dataset = tf_dataset(test_x, test_y, batch=batch_size)